In [7]:
import pyaudio

def list_audio_input_devices():
    p = pyaudio.PyAudio()
    device_count = p.get_device_count()
    for i in range(device_count):
        device_info = p.get_device_info_by_index(i)
        device_id = device_info['index']
        device_name = device_info['name']
        max_input_channels = device_info['maxInputChannels']
        # Display the device ID and name only if it has input channels (microphone)
        if max_input_channels > 0:
            print(f"Device ID: {device_id}, Device Name: {device_name}, Input Channels: {max_input_channels}")
    p.terminate()

# Call the function to list the audio input devices (microphones)
list_audio_input_devices()

Device ID: 3, Device Name: HD-Audio Generic: ALC257 Analog (hw:1,0), Input Channels: 2
Device ID: 4, Device Name: acp: - (hw:2,0), Input Channels: 2
Device ID: 5, Device Name: ThinkPad USB-C Dock Gen2 USB Au: Audio (hw:3,0), Input Channels: 1
Device ID: 6, Device Name: P2418HZm: USB Audio (hw:5,0), Input Channels: 2
Device ID: 8, Device Name: pulse, Input Channels: 32
Device ID: 9, Device Name: default, Input Channels: 32


In [ ]:
import pyaudio
import numpy as np
import threading
import openai
import keyring
import wave
import tempfile
import time
from datetime import datetime

# Define audio recording parameters
input_default_samplerate = 44100  # Sample rate
device_id = 9  # Change this to the appropriate audio device ID
recording_duration = 5  # Duration of recording for noise baseline in seconds
chunk_duration = 3  # Duration of each chunk in seconds
chunk_samples = int(input_default_samplerate * chunk_duration)  # Number of samples in each chunk
threshold_multiplier = 1.5  # Multiplier for noise threshold
gain_factor = 2.0  # Amplify the audio by a factor of 2

# Define a function to save the numpy array as a WAV file in memory
def save_wav_in_memory(audio_data):
    byte_io = io.BytesIO()
    with wave.open(byte_io, 'wb') as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(input_default_samplerate)
        wav_file.writeframes(audio_data.astype(np.int16).tobytes())
    byte_io.seek(0)
    return byte_io.read()

# Calculate root-mean-square (RMS) energy of a chunk
def calculate_rms(chunk):
    return np.sqrt(np.mean(np.square(chunk)))

# Simple Voice Activity Detection (VAD)
def is_voice_active(chunk, threshold):
    rms = calculate_rms(chunk)
    return rms, rms > threshold

# Define a function for recording audio and transcribing chunks
def record_and_transcribe(stop_event, threshold):
    active_chunk = np.array([])
    is_talking = False
    recording_start = time.time()

    def callback(in_data, frame_count, time_info, status):
        nonlocal active_chunk, is_talking, recording_start
        # Convert byte data to numpy array and flatten
        chunk = np.frombuffer(in_data, dtype=np.float32).flatten()
        # Apply gain factor
        chunk = chunk * gain_factor
        rms, voice_active = is_voice_active(chunk, threshold)
        if voice_active:
            if not is_talking:
                is_talking = True
                recording_start = time.time()
                print("Started Talking", rms, threshold)
            else:
                print("Is Talking", rms, threshold, time.time() - recording_start)
            # If active voice detected, append chunk to active_chunk
            active_chunk = np.concatenate((active_chunk, chunk))
        elif len(active_chunk) > 0:
            print("Stopped Talking", rms, threshold, time.time() - recording_start)
            # Save and transcribe the active_chunk
            audio_bytes = save_wav_in_memory(active_chunk)
            datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            temp_filename = f"temp_audio_{datetime_str}.wav"
            with open(temp_filename, "wb") as f:
                f.write(audio_bytes)
            print('wav:', temp_filename)
            file = open(temp_filename, "rb")
            transcription = openai.Audio.transcribe("whisper-1", file)
            print(transcription)
            active_chunk = np.array([])
        return (in_data, pyaudio.paContinue)

    # Initialize PyAudio
    p = pyaudio.PyAudio()
    stream = p.open(
        format=pyaudio.paFloat32,
        channels=1,
        rate=input_default_samplerate,
        input=True,
        input_device_index=device_id,
        stream_callback=callback,
        frames_per_buffer=chunk_samples
    )  
    # Start the audio stream
    stream.start_stream()

    # Wait for stop_event to be set
    stop_event.wait()

    # Stop the audio stream
    stream.stop_stream()
    stream.close()

    # Terminate PyAudio
    p.terminate()

    print("Recording and transcription complete.")

# OpenAI API key
openai.api_key = keyring.get_password("system", "openai_key")

# Create an event to signal recording thread to stop
stop_event = threading.Event()

# Set an initial noise threshold
noise_threshold = 0.008

# Start recording and transcribing on a separate thread
recording_thread = threading.Thread(target=record_and_transcribe, args=(stop_event, noise_threshold))
recording_thread.start()

# Prompt user to press Enter to stop recording
input("Recording started. Press Enter to stop recording...\n")

# Signal the recording thread to stop
stop_event.set()
recording_thread.join()

print("Recording and transcription complete.")

Recording started. Press Enter to stop recording...
 


ALSA lib pcm_pulse.c:758:(pulse_prepare) PulseAudio: Unable to create stream: Timeout

Expression 'alsa_snd_pcm_prepare( stream->capture.pcm )' failed in 'src/hostapi/alsa/pa_linux_alsa.c', line: 2932
Expression 'AlsaStart( stream, 0 )' failed in 'src/hostapi/alsa/pa_linux_alsa.c', line: 4244


TypeError: record_and_transcribe() missing 2 required positional arguments: 'stop_event' and 'threshold'